# Quick and Dirty Removal of "Drive By" users. 
A "drive by" user is one who logged in once but never did anything with AGOL.  They have no content, are not a member of any groups, have not logged in over the last year, and use no storage.  In other words, they can be safely deleted to reclaim named users in an auto-provisioning organization because when they come back, it will be like they never left.  In order to get rid of these users, we need to
1. Identify them based on the criteria above
2. Remove all their licenses, releasing them to the pool
3. Delete them
### Author: Seth Peery, Virginia Polytechnic Institute and State University
### Last Modified July 3, 2019

In [5]:
from arcgis.gis import GIS
import requests
import time
import csv
import json
import pandas
from time import strftime
from urllib.parse import urlparse
import getpass

In [6]:
# Make the connection to your ArcGIS Online Organization
orgURL = "https://virginiatech.maps.arcgis.com" # <==update for your org
orgUser = "sspeery"                             # <==update for your org
orgPwd = getpass.getpass("Enter your AGOL Administrator Password")
gis = GIS(orgURL,username=orgUser,password=orgPwd) 
orgShortName = urlparse(orgURL).hostname[:-len('.maps.arcgis.com')]

# verify that it works
try:
    org = gis.properties.name
    print ("Connected to " + org)
except exception as ex:
        print ("Error retrieving AGOL org properties.")

Enter your AGOL Administrator Password········
Connected to Virginia Tech


In [8]:
# Do a search of all our users.
# The idea is that we only want to query the portal once, then do the rest offline.
# Note that the variable max_users must be >= the number of users in the org to get everybody
# Once this search is done we get a notebook-global data structure (a pandas df) that we can use for subsequent queries
# This takes a long time to run if you have a lot of users in your organization, but you only need to run this once. 

userList = []
users = gis.users.search(max_users=9999)

for user in users:
   
    #These things come straight from the user dict
    d_esriUsername = user.username
    d_fullName = user.fullName
    d_email = user.email
    d_role = user.role
    d_storage = (user.storageUsage / 1024)
    
    #number of content items <=100 is returned by length of items arr
    d_items = len(user.items())
    #print(d_items)
    
    #VT PID is returned by stripping off the _virginiatech
    d_pid = user.username.rsplit("_"+orgShortName)[0]
    
    #last access comes from https://developers.arcgis.com/python/guide/accessing-and-managing-users/
    t_last_accessed = time.localtime(user.lastLogin/1000)
    d_lastAccess = "{}/{}/{}".format(t_last_accessed[0], t_last_accessed[1], t_last_accessed[2])
    
    #count of groups this user is a member of
    d_groupCount = len(user.groups)
    
    #Now build a data structure    
    currentUserInfo = {"pid":d_pid,
                        "esriUsername":d_esriUsername,
                        "fullName":d_fullName,
                        "email":d_email,
                        "storage":d_storage,
                        "role":d_role,
                        "lastAccess":d_lastAccess,
                        "groups":d_groupCount,
                        "items":d_items}
    userList.append(currentUserInfo)
    
# iteration done.
# now let's make a dataframe.  We'll use this later.
df = pandas.DataFrame(userList)

# Next, filter out the Enterprise Logins to create a new data structure 
# because we want to exempt legacy ESRI global accounts and service accounts
# therefore this operates only on our Enterprise Logins.
# NOTE that you need to adjust this to your environment - both your org_short_name and its length (+1 for the underscore)
vtEnterpriseLogins = df.loc[df['esriUsername'].str[-(len("_"+orgShortName)):] == "_"+orgShortName]
vtEnterpriseLogins



,email,esriUsername,fullName,groups,items,lastAccess,pid,role,storage
0,,aabhi_virginiatech,Abhishek Abhyankar,1,14,2016/5/4,aabhi,org_publisher,1399.333008
1,aaronh99@vt.edu,aaronh99_virginiatech,Aaron Henry,1,12,2018/12/5,aaronh99,org_publisher,14339.505859
2,aarony98@vt.edu,aarony98_virginiatech,Aaron Yowell,1,6,2019/2/18,aarony98,org_publisher,16128.606445
3,aarshadi@vt.edu,aarshadi_virginiatech,,0,6,2019/5/9,aarshadi,org_publisher,11786.054688
4,,abby15_virginiatech,Abby Qualliotine,1,2,2016/4/5,abby15,wEn8NO3VNLcycmpV,2006.706055
5,abbyb99@vt.edu,abbyb99_virginiatech,Abigail Bryerton,1,0,2018/3/11,abbyb99,org_publisher,0.000000
6,abbye97@vt.edu,abbye97_virginiatech,Abigail England,2,8,2019/4/1,abbye97,org_publisher,3674.511719
7,abbyp98@vt.edu,abbyp98_virginiatech,Abigail Potter,1,15,2018/5/1,abbyp98,org_publisher,42183.235352
8,abcazenave@vt.edu,abcazenave_virginiatech,Alexandre Cazenave,0,0,2019/5/6,abcazenave,org_publisher,0.000000
9,abdela98@vt.edu,abdela98_virginiatech,Abdel Rahman Altarazi,0,0,2018/10/1,abdela98,org_publisher,0.000000


## "Drive by" user identification
####  I feel reasonably confident we can get rid of users who 
* own no content items
* are members of no groups
* use no storage
* have not logged in for a year  (or, precisely, have not logged in in 2018 or 2019 - you could set this to be more aggressive if needed)

... since if that user logs back in, it will be like they never left.

In [10]:
# From a dataframe of VT Enterprise Logins only, create a dataframe that meets the above criteria.
deleteList = vtEnterpriseLogins.loc[(vtEnterpriseLogins['storage'] == 0) & (vtEnterpriseLogins['items'] == 0) &(vtEnterpriseLogins['groups'] ==0) &(vtEnterpriseLogins['lastAccess'].str[:4] < '2018')]
deleteList


,email,esriUsername,fullName,groups,items,lastAccess,pid,role,storage
14,abreed2@vt.edu,abreed2_virginiatech,Alyson Breeding,0,0,2016/9/8,abreed2,org_publisher,0.0
20,acorrea@vt.edu,acorrea_virginiatech,Angela Correa-Becker,0,0,2017/9/25,acorrea,org_publisher,0.0
37,aguest@vt.edu,aguest_virginiatech,Alex Guest,0,0,2017/4/7,aguest,org_publisher,0.0
48,ajwells@vt.edu,ajwells_virginiatech,Alex Wells,0,0,2016/12/20,ajwells,org_publisher,0.0
61,alena5@vt.edu,alena5_virginiatech,Alena Deveau,0,0,2017/4/20,alena5,org_publisher,0.0
74,alext10@vt.edu,alext10_virginiatech,Alex Tucker,0,0,2016/9/6,alext10,org_publisher,0.0
80,aliyahli@vt.edu,aliyahli_virginiatech,Aliyah Linatoc,0,0,2016/9/8,aliyahli,org_publisher,0.0
126,andman@vt.edu,andman_virginiatech,Andrew Hoffman,0,0,2017/4/14,andman,org_publisher,0.0
155,apagan8@vt.edu,apagan8_virginiatech,Alexis Pagan,0,0,2016/7/26,apagan8,org_publisher,0.0
181,asp2340@vt.edu,asp2340_virginiatech,Andrew Pierce,0,0,2016/7/25,asp2340,KeUYufHBaBEQkoZq,0.0


In [13]:

# Optionally, Serialize the list of users who are candidates for deletion so we can take a look at it in Excel.  (Out of band human inspection)
deleteList.to_csv('driveby-'+strftime("%Y%m%d-%H%M%S",time.localtime())+".csv")


## Deprovision all user licenses and entitlements
Before we can actually delete a user we have to get rid of all their entitlements, releasing all add-on licenses. 
We can either do this by attempting to revoke every entitement in the org or by inspecting the specific user via the REST API to get their actual entitlements and just revoking what they have.

See https://github.com/sspeery/educ2019/blob/master/GetOrganizationEntitlements.ipynb

In [9]:
# Retrieve all the entitlements in the org
lics = gis.admin.license.all()
licensedProducts = []   # array of the specific Products for which there may be entitlements
licenseArr = []         # array of the actual licenses for the Products
for lic in lics:
    try:
        licensedProducts.append(json.loads(str(lic.properties))['listing']['title'])
    except KeyError:
        pass

entDf = pandas.DataFrame()  # this stores your organization's entitlements total/assigned/remaining for each product in a dataframe
for product in licensedProducts:
    try:
        theLicense = gis.admin.license.get(product)
        licenseArr.append(theLicense)
        thisProductReportDf = theLicense.report #gets the total/assigned/remaining licenses for each product
        thisProductReportDf['Product'] = product
        entDf = entDf.append(thisProductReportDf)
    except KeyError:
        pass
    
# Create a dictionary of the form used by the UMich license assignment script
# See also: https://gist.github.com/sspeery/190d6dc45bc8f4c4b3ba0e2705850463
entitlements = dict.fromkeys(entDf['Product'].unique())
for key in entitlements:
    entitlements[key]=[]
for index, row in entDf.iterrows():
    entitlements[row['Product']].append(row['Entitlement'])
entitlements

{'Insights for ArcGIS': ['Insights'],
 'ArcGIS Pro': ['3DAnalystN',
  'airportsN',
  'businessStdN',
  'dataInteropN',
  'dataReviewerN',
  'defenseN',
  'desktopAdvN',
  'geostatAnalystN',
  'imageAnalystN',
  'LocRefDesktopN',
  'maritimeN',
  'networkAnalystN',
  'productionMapN',
  'publisherN',
  'spatialAnalystN',
  'workflowMgrN'],
 'GeoPlanner for ArcGIS': ['GeoPlanner'],
 'ArcGIS Community Analyst': ['CommunityAnlyst'],
 'Navigator for ArcGIS': ['Navigator'],
 'ArcGIS Business Analyst Web and Mobile Apps': ['BusinessAnlyst']}

In [14]:
# Now for each user in the deletion list we need to try and revoke each entitlement for everything they might have
justTheDriveByUserNames = deleteList[deleteList.columns[1]]

# Retrieve all the entitlements in the org
lics = gis.admin.license.all()
licensedProducts = []   # array of the specific Products for which there may be entitlements
licenseArr = []         # array of the actual licenses for the Products
for lic in lics:
    try:
        licensedProducts.append(json.loads(str(lic.properties))['listing']['title'])
    except KeyError:
        pass
for sUserToDelete in justTheDriveByUserNames:
    print ("Removing Entitlements for user " + sUserToDelete+"...")
    for product in licensedProducts:
        try: 
            print ("-"+ product)
            theLicense = gis.admin.license.get(product)
            theLicense.revoke(username=sUserToDelete, entitlements='*')
        except:
            print ("I'm just not worrying about this right now")
        
    

Removing Entitlements for user abreed2_virginiatech...
-Insights for ArcGIS
-ArcGIS Pro
-Drone2Map for ArcGIS
-GeoPlanner for ArcGIS
-AppStudio for ArcGIS
-ArcGIS Community Analyst
-Navigator for ArcGIS


not licensed by user


-Admin Tools for ArcGIS℠ Online
I'm just not worrying about this right now
-ArcGIS Business Analyst Web and Mobile Apps
Removing Entitlements for user acorrea_virginiatech...
-Insights for ArcGIS
-ArcGIS Pro
-Drone2Map for ArcGIS
-GeoPlanner for ArcGIS
-AppStudio for ArcGIS
-ArcGIS Community Analyst


not licensed by user


-Navigator for ArcGIS
-Admin Tools for ArcGIS℠ Online
I'm just not worrying about this right now
-ArcGIS Business Analyst Web and Mobile Apps
Removing Entitlements for user aguest_virginiatech...
-Insights for ArcGIS
-ArcGIS Pro
-Drone2Map for ArcGIS
-GeoPlanner for ArcGIS
-AppStudio for ArcGIS
-ArcGIS Community Analyst
-Navigator for ArcGIS


...

-Navigator for ArcGIS
-Admin Tools for ArcGIS℠ Online
I'm just not worrying about this right now
-ArcGIS Business Analyst Web and Mobile Apps
Removing Entitlements for user zdaley24_virginiatech...
-Insights for ArcGIS
-ArcGIS Pro
-Drone2Map for ArcGIS
-GeoPlanner for ArcGIS
-AppStudio for ArcGIS
-ArcGIS Community Analyst


not licensed by user


-Navigator for ArcGIS
-Admin Tools for ArcGIS℠ Online
I'm just not worrying about this right now
-ArcGIS Business Analyst Web and Mobile Apps
Removing Entitlements for user zeman97_virginiatech...
-Insights for ArcGIS
-ArcGIS Pro
-Drone2Map for ArcGIS
-GeoPlanner for ArcGIS
-AppStudio for ArcGIS
-ArcGIS Community Analyst


not licensed by user


-Navigator for ArcGIS
-Admin Tools for ArcGIS℠ Online
I'm just not worrying about this right now
-ArcGIS Business Analyst Web and Mobile Apps


In [20]:
# To actually delete the user we need a user object, so we will use the portal method gis.users.search() with a username (_virginiatech)  as input.
for sUserToDelete in justTheDriveByUserNames:
    print ("Deleting " + sUserToDelete +"...")
    # right now we just make a roundtrip to the server for this reference
    userObjectToDelete = gis.users.search(sUserToDelete, max_users=1)
    userObjectToDelete[0].delete()
    print ("    Deprovisioned "+ userObjectToDelete[0].username)

Deleting abreed2_virginiatech...
    Deprovisioned abreed2_virginiatech
Deleting acorrea_virginiatech...
    Deprovisioned acorrea_virginiatech
Deleting aguest_virginiatech...
    Deprovisioned aguest_virginiatech
Deleting ajwells_virginiatech...
    Deprovisioned ajwells_virginiatech
Deleting alena5_virginiatech...
    Deprovisioned alena5_virginiatech
Deleting alext10_virginiatech...
    Deprovisioned alext10_virginiatech
Deleting aliyahli_virginiatech...
    Deprovisioned aliyahli_virginiatech
Deleting andman_virginiatech...
    Deprovisioned andman_virginiatech
Deleting apagan8_virginiatech...
    Deprovisioned apagan8_virginiatech
Deleting asp2340_virginiatech...
    Deprovisioned asp2340_virginiatech
Deleting augusta_virginiatech...
    Deprovisioned augusta_virginiatech
Deleting aupdike_virginiatech...
    Deprovisioned aupdike_virginiatech
Deleting averill1_virginiatech...
    Deprovisioned averill1_virginiatech
Deleting bbritt1_virginiatech...
    Deprovisioned bbritt1_virgini

    Deprovisioned jb1013_virginiatech
Deleting jconnorh_virginiatech...
    Deprovisioned jconnorh_virginiatech
Deleting jcy2123_virginiatech...
    Deprovisioned jcy2123_virginiatech
Deleting jdavis23_virginiatech...
    Deprovisioned jdavis23_virginiatech
Deleting jdh15_virginiatech...
    Deprovisioned jdh15_virginiatech
Deleting jfike_virginiatech...
    Deprovisioned jfike_virginiatech
Deleting jlac13_virginiatech...
    Deprovisioned jlac13_virginiatech
Deleting jmcg_virginiatech...
    Deprovisioned jmcg_virginiatech
Deleting jmkane_virginiatech...
    Deprovisioned jmkane_virginiatech
Deleting jmmays13_virginiatech...
    Deprovisioned jmmays13_virginiatech
Deleting jmr308_virginiatech...
    Deprovisioned jmr308_virginiatech
Deleting joeyc13_virginiatech...
    Deprovisioned joeyc13_virginiatech
Deleting joshp13_virginiatech...
    Deprovisioned joshp13_virginiatech
Deleting josiahde_virginiatech...
    Deprovisioned josiahde_virginiatech
Deleting josies8_virginiatech...
    D

    Deprovisioned tmcavoy_virginiatech
Deleting trevor8_virginiatech...
    Deprovisioned trevor8_virginiatech
Deleting vaneb2_virginiatech...
    Deprovisioned vaneb2_virginiatech
Deleting vkrist1_virginiatech...
    Deprovisioned vkrist1_virginiatech
Deleting vmatth7_virginiatech...
    Deprovisioned vmatth7_virginiatech
Deleting vmr269_virginiatech...
    Deprovisioned vmr269_virginiatech
Deleting vtlakoba_virginiatech...
    Deprovisioned vtlakoba_virginiatech
Deleting wadeo7_virginiatech...
    Deprovisioned wadeo7_virginiatech
Deleting wdavid5_virginiatech...
    Deprovisioned wdavid5_virginiatech
Deleting weaverea_virginiatech...
    Deprovisioned weaverea_virginiatech
Deleting wesj2363_virginiatech...
    Deprovisioned wesj2363_virginiatech
Deleting wjack92_virginiatech...
    Deprovisioned wjack92_virginiatech
Deleting wkait14_virginiatech...
    Deprovisioned wkait14_virginiatech
Deleting wrachael_virginiatech...
    Deprovisioned wrachael_virginiatech
Deleting wyuri91_virgin